# Combining scheduled and actual headway calcs

In [1]:

from headways import *

In [2]:
# Values to use for testing
gtfs_version_id = '20230105'

In [3]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id) 

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.40it/s]


# Get headway stats for an entire route on a single day
Takes 10 minutes on my computer...

In [7]:
%%capture --no-display

stats_all_stops_55_20230118 = get_stats_all_stops(gtfs_feed, '55', '2023-01-18')

In [11]:
# plot route 55 linestring and stops with headway data

linestring_55 = gpd.read_file('headway_summaries/route55_linestring.json')
stops_55 = gpd.read_file('headway_summaries/route55_2023-01-18.json')

m = linestring_55.explore(color='#41B6E6', tiles="CartoDB positron", )
stops_55.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))


In [12]:
%%capture --no-display

stats_all_stops_55_20230122 = get_stats_all_stops(gtfs_feed, '55', '2023-01-22')

TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and int

# This worked for route 55 on Wed. 1/18/2023.  Not working on Sunday 1/22/2023.
### Next steps:  figure out why

In [13]:
## Scratch 

vehicles = get_chn_vehicles('2023-01-22')

vehicles

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date
0,7994,2023-01-22 00:02:00+00:00,41.895691,-87.620377,180,18414,3,Michigan/Chicago,70324,False,153717,238695847,3 -715,NaN,bus_data/2023-01-22/00:02:56.json,2023-01-22 00:02:00,0,2023-01-22
1,7923,2023-01-22 00:02:00+00:00,41.893933,-87.619962,178,18414,3,Michigan/Chicago,71059,True,153716,238696489,3 -705,NaN,bus_data/2023-01-22/00:02:56.json,2023-01-22 00:02:00,0,2023-01-22
2,1255,2023-01-22 00:02:00+00:00,41.868749,-87.624199,180,18415,3,95th/RED LINE,13122,False,1051752,238696081,3 -720,NaN,bus_data/2023-01-22/00:02:56.json,2023-01-22 00:02:00,0,2023-01-22
3,7996,2023-01-22 00:02:00+00:00,41.802012,-87.616196,170,18415,3,95th/RED LINE,39037,False,1051751,238695851,3 -719,NaN,bus_data/2023-01-22/00:02:56.json,2023-01-22 00:02:00,0,2023-01-22
4,1272,2023-01-22 00:02:00+00:00,41.766840,-87.615318,178,18415,3,95th/RED LINE,51905,False,1051750,238697401,3 -703,NaN,bus_data/2023-01-22/00:02:56.json,2023-01-22 00:02:00,0,2023-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164953,4351,2023-01-23 23:57:00+00:00,41.913914,-87.631741,181,8102,151,Union Station,40851,False,398,238769795,151 -524,NaN,bus_data/2023-01-23/23:57:56.json,2023-01-23 23:57:00,23,2023-01-23
164954,1882,2023-01-23 23:57:00+00:00,41.880730,-87.624467,178,8102,151,Union Station,54824,False,400,238769794,N22 -593,NaN,bus_data/2023-01-23/23:57:56.json,2023-01-23 23:57:00,23,2023-01-23
164955,1834,2023-01-23 23:57:00+00:00,42.007904,-87.665489,90,904,155,Kedzie,0,False,1005709,238770708,155 -551,NaN,bus_data/2023-01-23/23:57:56.json,2023-01-23 23:57:00,23,2023-01-23
164956,1719,2023-01-23 23:57:00+00:00,41.997519,-87.702121,267,904,155,Kedzie,16930,False,1005708,238771194,155 -507,NaN,bus_data/2023-01-23/23:57:56.json,2023-01-23 23:57:00,23,2023-01-23


In [14]:
## Scratch ##

def get_actual_headways(
    vehicles:pd.DataFrame, rt:str, stop_id:str, direction:str, 
    active_service_times:list) -> pd.DataFrame:

        '''
        Parameters:\n

        vehicles is a dataframe obtained using get_chn_vehicles().\n

        rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n

        stop_id is the stop id of a single bus stop as a string.\n

        direction is the direction of travel as a string: 'Northbound', 'Southbound', 
        'Eastbound', or 'Westbound'

        active_service_times is a list of start/end times when buses are 
        scheduled to be in service at a given stop. This list is generated
        by the get_active_service_times() function.\n

        Data returned:\n
        Columns are added to the vehicles dataframe indicating the start and end time
        and the start and end distances along a pattern for each interval where a bus
        passed a stop (start_time, end_time, start_pdist, end_pdist), the estimated time 
        each bus actually arrived at the stop (est_stop_time), and headway between each 
        bus (est_headway).  Direction of travel (rtdir) and stop id (stpid)
        are also included.\n
        Headways are not calculated for the first bus in each active service period.  This
        ensures that regularly scheduled out-of-service periods do not appear as long headways.
        '''

        df_output = pd.DataFrame()

        # Times buses stopped at each stop on the route
        df_stoptimes = get_actual_stoptimes(rt, vehicles).copy()

        # Filter to buses stopping at the specified stop in the specified direction
        df_stop_direction = df_stoptimes.loc[
            (df_stoptimes['stpid'] == stop_id)]
            # & (df_stoptimes['rtdir'] == direction)
            # ]

        # Consider each active service timeframe
        for start_time, end_time in list(zip(
            active_service_times['start_time'],
            active_service_times['end_time'])):

            print(f'{start_time} to {end_time}')


            # filter for stops happening after the service time start
            df_stop_direction_servicetime = df_stop_direction.loc[
                df_stop_direction['est_stop_time'] >= start_time
                ]

            # filter for stops happening before the service time end
            df_stop_direction_servicetime = df_stop_direction_servicetime.loc[
                df_stop_direction_servicetime['est_stop_time'] <= end_time
                ]

            # print(df_stop_direction_servicetime)

            # Sort chronologically
            df_stop_direction_servicetime.sort_values(by='est_stop_time',ascending=True, inplace=True)

            # list stop times in chronological order
            stop_times = df_stop_direction_servicetime['est_stop_time'].tolist()

            # calculate previous stop time for each line
            prev_stop_times = np.roll(stop_times,1)
            df_stop_direction_servicetime['previous_stop_time'] = prev_stop_times

            # calculate headway
            df_stop_direction_servicetime['est_headway'] = (
                df_stop_direction_servicetime['est_stop_time'] 
                - df_stop_direction_servicetime['previous_stop_time']
                )

            # drop previous stop time column, no longer needed
            df_stop_direction_servicetime = df_stop_direction_servicetime.drop(
                    'previous_stop_time', axis=1)

            # Remove headway from the first bus in the dataset since we don't have the 
            # previous bus to compare with
            df_stop_direction_servicetime['est_headway'].iloc[0] = None
                
            df_output = pd.concat([df_output, df_stop_direction_servicetime])

        return df_output